In [1]:
access_token='1248089353017905152-YoPpxXLfW6rFpVsDnLJtXOKvqSwFPQ'
access_token_secret='fduHdyl1Z3pr5FXPpXy38uZQ0GDAKTQGCErTpIhOQaYyO'
consumer_key='KGy6h1JLWwYNASj926qpAk7Hi'
consumer_secret='37qeEcAKDl846VCTgIgZAdJWna4p9XKMZTgd86b2QzYBVni45z'

In [3]:
#from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import tweepy
from kafka import KafkaProducer

producer = KafkaProducer(bootstrap_servers='localhost:9092') #Same port as your Kafka server


topic_name = "covid"


class twitterAuth():
    """SET UP TWITTER AUTHENTICATION"""

    def authenticateTwitterApp(self):
        auth = OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)

        return auth



class TwitterStreamer():

    """SET UP STREAMER"""
    def __init__(self):
        self.twitterAuth = twitterAuth()

    def stream_tweets(self):
        while True:
            listener = ListenerTS() 
            auth = self.twitterAuth.authenticateTwitterApp()
            stream = Stream(auth, listener)
            stream.filter(track=["covid","vaccine","thirdwave","spreading","omicron"], stall_warnings=True, languages= ["en"])


class ListenerTS(tweepy.StreamListener):

    def on_data(self, raw_data):
            producer.send(topic_name, str.encode(raw_data))
            #print(str.encode(raw_data))
            return True


if __name__ == "__main__":
    TS = TwitterStreamer()
    TS.stream_tweets()

KeyboardInterrupt: 

In [ ]:
#from kafka import KafkaConsumer
#import json

#topic_name = 'trump'

#consumer = KafkaConsumer(
#    topic_name,
#     bootstrap_servers=['localhost:9092'],
#     auto_offset_reset='latest',
#     enable_auto_commit=True,
#     auto_commit_interval_ms =  5000,
#     fetch_max_bytes = 128,
#     max_poll_records = 100,

#     value_deserializer=lambda x: json.loads(x.decode('utf-8')))

#for message in consumer:
# tweets = json.loads(json.dumps(message.value))
# print(tweets)

##customize producer

In [ ]:
import gensim

In [ ]:
import tweepy
import time
from datetime import datetime
from kafka import KafkaProducer,KafkaConsumer,KafkaClient

auth=tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api=tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

def normalize_timestamp(time):
    mytime=datetime.strptime(time,'%Y-%m-%d %H:%M:%S')
    return(mytime.strftime('%Y-%m-%d %H:%M:%S'))
producer = KafkaProducer(bootstrap_servers='localhost:9092')
topic_name="covid"
search_words = ["#covid19", "omicron", "lockdown"]
def get_twitter_data():
    res=api.search(q="omicron",count=100,tweet_mode='extended')
    #tweets=tweepy.Cursor(api.search,search_words,lang='en').items(10)
    for i in res:
        record=""
        record+=str(i.user.id_str)
        record+=";"
        record+=str(normalize_timestamp(str(i.created_at)))
        record+=";"
        record+=str(i.user.followers_count)
        record+=";"
        record+=str(i.user.location)
        record+=";"
        record+=str(i.favorite_count)
        record+=";"
        record+=str(i.retweet_count)
        record+=";"
        record+=str(i.full_text.encode('UTF-8'))
        producer.send(topic_name,str.encode(record))

get_twitter_data()

def periodic_work(interval):
    while True:
        get_twitter_data()
        time.sleep(interval)

periodic_work(60*0.1)


In [ ]:
import tweepy
import time
from datetime import datetime
from kafka import KafkaProducer,KafkaConsumer,KafkaClient

auth=tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api=tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

def normalize_timestamp(time):
    mytime=datetime.strptime(time,'%Y-%m-%d %H:%M:%S')
    return(mytime.strftime('%Y-%m-%d %H:%M:%S'))
producer = KafkaProducer(bootstrap_servers='localhost:9092')
topic_name="covid"

def get_twitter_data():
    res=api.search("covid or vaccine or omicron")
    for i in res:
        print(i)

get_twitter_data()

def periodic_work(interval):
    while True:
        get_twitter_data()
        time.sleep(interval)

periodic_work(60*0.1)


##customize Tweeter collection send to kafka_Consumer and pyspark consumer

In [4]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to C:\Users\Mrinmoy
[nltk_data]     Sadhukhan\AppData\Roaming\nltk_data...


True

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
import re # for regular expressions
import string

In [3]:
import os
os.environ.update({'MALLET_HOME':r'C:/mallet-2.0.8/'})
mallet_path = 'C:/mallet-2.0.8/bin/mallet' # update this path
import gensim

negativemodel = gensim.models.wrappers.LdaMallet.load('lada_negative_model.model')
positivemodel = gensim.models.wrappers.LdaMallet.load('lada_positive_model.model')
negativemodel.prefix='C:/mallet_temp/negative/' ##what does it mean?? is it searching  for mallet model
positivemodel.prefix='C:/mallet_temp/positive/'
positivemodel.mallet_path='C:/mallet-2.0.8/bin/mallet'
negativemodel.mallet_path='C:/mallet-2.0.8/bin/mallet'
import pickle
with open("negative_extended_query_set", "rb") as fp:
    negative_query = pickle.load(fp)
with open("positive_extended_query_set","rb") as fs:
    positive_query = pickle.load(fs)
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
import re # for regular expressions
import string
import gensim.corpora as corpora
id2word = corpora.Dictionary(negative_query) ### from here rochhio expansion system will entry !! may be
id2word1 = corpora.Dictionary(positive_query)
tokenizer = TweetTokenizer(preserve_case=False, 
                           strip_handles=True,
                           reduce_len=True)
from gensim.models import CoherenceModel
import pandas as pd
def format_topics_sentences(ldamodel, corpus, text):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num)]), ignore_index=True)
            else:
                break
    dominant_topic = topic_num

    # Add original text to the end of the output
    
    return(dominant_topic)
def subtopic_detection(key,text):
    text1= tokenizer.tokenize(text)
    if key == "Positive":
        corpus = [id2word1.doc2bow(text1)]
        df_topic_sents_keywords = format_topics_sentences(positivemodel, corpus, text)
    if key == "Negative":
        corpus = [id2word.doc2bow(text1)]
        df_topic_sents_keywords = format_topics_sentences(negativemodel, corpus, text)
    if key == "Neutral":
        df_topic_sents_keywords = 0.0
        
    ##total operation now will be here
    return df_topic_sents_keywords


In [4]:
tokenizer = TweetTokenizer(preserve_case=False, 
                           strip_handles=True,
                           reduce_len=True)
stopwords_english = stopwords.words('english')
def cleanstopword(tweet_tokens):
  tweets_clean = []
  for word in tweet_tokens:
    if (word not in stopwords_english and word not in string.punctuation):
      tweets_clean.append(''.join(word))
  return tweets_clean
stemmer = PorterStemmer()
def cleanstem(tweets_clean):
  # Create an empty list to store the stems
  tweets_stem = []
  for word in tweets_clean:
    stem_word = stemmer.stem(word)  # stemming word
    tweets_stem.append(stem_word)  # append to the list
  return tweets_stem

def cleantweets(text):
    text=str(text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'RT[\s]+', '', text)
    text = re.sub(r'https?:\/\/\S+', '', text)
    text = re.sub(r'[^a-zA-Z#]+', ' ',text)
    text = text.lower()
    text = tokenizer.tokenize(text)
    text = cleanstopword(text)
    text = cleanstem(text)
    text = TreebankWordDetokenizer().detokenize(text)
    return text


In [5]:
#from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import tweepy
from textblob import TextBlob
from kafka import KafkaProducer

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
producer = KafkaProducer(bootstrap_servers='localhost:9092') #Same port as your Kafka server


Input_topic_name ='quickstart-events'

sia=SIA()


    
class twitterAuth():
    """SET UP TWITTER AUTHENTICATION"""

    def authenticateTwitterApp(self):
        auth = OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)

        return auth



class TwitterStreamer():

    """SET UP STREAMER"""
    def __init__(self):
        self.twitterAuth = twitterAuth()

    def stream_tweets(self):
        while True:
            listener = StreamListener() 
            auth = self.twitterAuth.authenticateTwitterApp()
            stream = Stream(auth, listener)
            stream.filter(track=["covid","vaccine","thirdwave","spreading","omicron"], stall_warnings=True, languages= ["en"])


#class ListenerTS(tweepy.StreamListener):

#    def on_data(self, raw_data):
#            producer.send(topic_name, str.encode(raw_data))
            #print(str.encode(raw_data))
#            return True
class StreamListener(tweepy.StreamListener):
    def on_status(self, status):
        if status.retweeted:
            return

        description = status.user.description
        #loc = status.user.location
        text = status.text
        text = cleantweets(text)
        #coords = status.coordinates
        #geo = status.geo
        name = status.user.screen_name
        #user_created = status.user.created_at
        #followers = status.user.followers_count
        #id_str = status.id_str
        created = status.created_at
        #retweets = status.retweet_count
        #bg_color = status.user.profile_background_color
        #blob = TextBlob(text)
        #sent = blob.sentiment
        score = sia.polarity_scores(text)
        score=score['compound']
        if score>=0.05:
            sentiment='Positive'
        elif score<=(-0.05):
            sentiment='Negative'
        else:
            sentiment='Neutral'
        subtopic=subtopic_detection(sentiment,text)

        producer.send(Input_topic_name,str.encode(text),str.encode(str(subtopic)))

def periodic_work(interval):
    while True:
        TS = TwitterStreamer()
        TS.stream_tweets()
        #get_twitter_data()
        time.sleep(interval)

if __name__ == "__main__":
    periodic_work(60*0.1)
    

ProtocolError: ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))